# Process HPWREN Camera Metadata (Firemap API)

<b>Summary:</b><br>
Processes the hpwren only camera metadata pulled from firemap pylaski api.

<!-- - Get weather station metadata via synoptic api
- Save raw weather station metadata file
- Read in processed camera metadata
- Filter out weather stations to ensure they have at least 1 years worth of data for time period
- Map up to x stations to each camera
- Save camera-to-stations mapping file
 -->
<b>Output:</b><br>
.<br>
└── data<br>
&emsp;&emsp;&emsp;└── processed<br>
&emsp;&emsp;&emsp;&nbsp;&nbsp;&emsp;&emsp;&nbsp;└── camera_metadata_hpwren.csv<br>

<b>Areas for Improvement:</b><br>

## Read All Camera Metadata

In [112]:
import json
from ast import literal_eval
from datetime import datetime

import pandas as pd
import numpy as np
import pytz
import geopandas
from shapely.geometry import Polygon, LineString, Point

In [113]:
# literal_eval to read columns as lists
cameras_df = pd.read_csv(
    "../../data/raw/camera_metadata.csv",
    converters={
        "geometry.coordinates": literal_eval,
        "properties.latest-images": literal_eval,
    },
)

In [114]:
cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.is_patrol_mode,properties.description.lastupdate,properties.description.region,properties.description.fov,properties.description.activated_at,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# Camera breakdown
# all camera metadata either has hpwren or axis prefix
cameras_df["properties.description.id"].str.split(r"_|-", expand=True)[0].str.extract(
    r"([a-zA-Z]+)"
)[0].str.lower().value_counts()

axis      1055
hpwren     103
Name: 0, dtype: int64

## Filter for HPWREN cameras only

In [116]:
# just get hpwren cameras (ignore axis)

hpwren_cameras_df = cameras_df[
    cameras_df["properties.description.id"]
    .str.lower()
    .str.contains("hpwren", regex=False)
].copy()

# dictoinary example below:
# hpwren_cameras = []
# for feat in features:
#     feat_id = feat["properties"]["description"]["id"]
#     if "hpwren" in feat_id.lower():
#         # print(feat_id)
#         hpwren_cameras.append(feat)

In [117]:
len(hpwren_cameras_df)

103

## Transformations

### Extract direction

In [118]:
# Extract direction from description
hpwren_cameras_df["direction"] = hpwren_cameras_df[
    "properties.description.id"
].str.split("_", n=1, expand=True)[1]

In [119]:
hpwren_cameras_df["direction"].value_counts()

north                26
east                 26
south                25
west                 25
unknown direction     1
Name: direction, dtype: int64

### Extract image name

In [120]:
# image name
hpwren_cameras_df["image_id"] = (
    hpwren_cameras_df["properties.latest-images"]
    .str[0]
    .str[0]
    .str["image"]
    .str.split("/")
    .str[-1]
    .str.split(".")
    .str[0]
)

In [121]:
hpwren_cameras_df.shape

(103, 31)

In [122]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.region,properties.description.fov,properties.description.activated_at,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current,direction,image_id
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c


### Extract long, lat, elevation

In [123]:
hpwren_cameras_df["long"] = hpwren_cameras_df["geometry.coordinates"].str[0]
hpwren_cameras_df["lat"] = hpwren_cameras_df["geometry.coordinates"].str[1]
hpwren_cameras_df["elevation"] = hpwren_cameras_df["geometry.coordinates"].str[2]

In [124]:
hpwren_cameras_df.shape

(103, 34)

In [125]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current,direction,image_id,long,lat,elevation
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c,-117.1200,32.5500,10
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c,-116.8081,33.1599,4055
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c,-116.8081,33.1599,4055
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c,-116.8081,33.1599,4055
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c,-116.8081,33.1599,4055


### Merge with manually entered camera metadata

- If manual data missing camera_name then merge
- If manual data has camera_name then union

In [126]:
cam_man_df = pd.read_csv("../../data/raw/camera_metadata_manual.csv")
cam_man_df = cam_man_df.add_suffix("_manual")

In [127]:
cam_man_df.shape

(131, 11)

In [128]:
cam_man_df.head()

,camera_id_manual,image_id_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual,center_lat_manual,center_long_manual
0,hpwren0_unknown direction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hpwren1_north,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.181599,-116.807554
2,hpwren1_east,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.158781,-116.790230
3,hpwren1_south,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.157932,-116.807962
4,hpwren1_west,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.159091,-116.858706


In [129]:
cam_man_na_df = cam_man_df[cam_man_df["camera_name_manual"].isna()]

In [130]:
cam_man_na_df.shape

(103, 11)

In [131]:
hpwren_cameras_df = hpwren_cameras_df.merge(
    cam_man_na_df,
    left_on="properties.description.id",
    right_on="camera_id_manual",
    how="inner",
)

In [132]:
hpwren_cameras_df.shape

(103, 45)

In [133]:
cam_man_nona_df = cam_man_df[~cam_man_df["camera_name_manual"].isna()]

In [134]:
cam_man_nona_df.shape

(28, 11)

In [135]:
hpwren_cameras_df = pd.concat([hpwren_cameras_df, cam_man_nona_df])

In [136]:
hpwren_cameras_df.shape

(131, 45)

In [137]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,image_id_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual,center_lat_manual,center_long_manual
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.181599,-116.807554
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.158781,-116.790230
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.157932,-116.807962
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.159091,-116.858706


In [138]:
# If original value null, then fill with new value

hpwren_cameras_df["camera_id"] = hpwren_cameras_df["properties.description.id"].fillna(
    hpwren_cameras_df["camera_id_manual"]
)

hpwren_cameras_df["camera_name"] = hpwren_cameras_df[
    "properties.description.name"
].fillna(hpwren_cameras_df["camera_name_manual"])

hpwren_cameras_df["image_id"] = hpwren_cameras_df["image_id"].fillna(
    hpwren_cameras_df["image_id_manual"]
)

hpwren_cameras_df["direction"] = hpwren_cameras_df["direction"].fillna(
    hpwren_cameras_df["direction_manual"]
)

hpwren_cameras_df["elevation"] = hpwren_cameras_df["elevation"].fillna(
    hpwren_cameras_df["elevation_manual"]
)

# If manual value null, then fill with orig value

hpwren_cameras_df["lat"] = hpwren_cameras_df["gmap_lat_manual"].fillna(
    hpwren_cameras_df["lat"]
)

hpwren_cameras_df["long"] = hpwren_cameras_df["gmap_long_manual"].fillna(
    hpwren_cameras_df["long"]
)

### Centerline Offset Angle Calculation & Camera Abbreviations

In [139]:
def find_cl_angle(ax, ay, bx, by, direction):
    a = np.array([ax, ay])
    b = np.array([bx, by])
    if direction.lower() in ['north', 'east', 'south', 'west']:
        if direction.lower() == 'north':
            cx = bx
            cy = by + 0.05
            if ax >= bx:
                sign = -1
            else:
                sign = 1
        if direction.lower() == 'east':
            cx = bx + 0.05
            cy = by
            if ay >= by:
                sign = 1
            else:
                sign = -1
        if direction.lower() == 'south':
            cx = bx
            cy = by - 0.05
            if ax >= bx:
                sign = 1
            else:
                sign = -1
        if direction.lower() == 'west':
            cx = bx - 0.05
            cy = by
            if ay >= by:
                sign = -1
            else:
                sign = 1

        c = np.array([cx, cy])

        ba = a - b
        bc = c - b

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle)
        return angle * sign
    else:
        return np.nan

In [141]:
hpwren_cameras_df['center_angle'] = hpwren_cameras_df.apply(lambda x: find_cl_angle( x['center_long_manual'], x['center_lat_manual'], x['long'], x['lat'], x['direction']), axis=1)
hpwren_cameras_df['camera_abbrev'] = hpwren_cameras_df.image_id.apply(lambda x: pd.Series(str(x).split("-")[0]))
hpwren_cameras_df = hpwren_cameras_df.reset_index(drop=True)

In [163]:
#x=.2222
#haversine((33.1599, -116.8081), (33.1599+x, -116.8081+x), unit=Unit.MILES)

def find_fov_right(direction, lat, long, fill_dist):
    #fill_dist = .2777 #25 miles away
    if direction == "north":
        return (long+fill_dist, lat+fill_dist)
    if direction == "east":
        return (long+fill_dist, lat-fill_dist)
    if direction == "south":
        return (long-fill_dist, lat-fill_dist)
    if direction == "west":
        return (long-fill_dist, lat+fill_dist)
    
def find_fov_left(direction, lat, long, fill_dist):
    #fill_dist = .2777 #25 miles away
    if direction == "north":
        return (long-fill_dist, lat+fill_dist)
    if direction == "east":
        return (long+fill_dist, lat+fill_dist)
    if direction == "south":
        return (long+fill_dist, lat-fill_dist)
    if direction == "west":
        return (long-fill_dist, lat-fill_dist)

In [164]:
fill_dist = .2777 #25miles out
fov_right = hpwren_cameras_df.apply(lambda x: find_fov_right(x['direction'], x['lat'], x['long'], fill_dist), axis=1)
fov_left = hpwren_cameras_df.apply(lambda x: find_fov_left(x['direction'], x['lat'], x['long'], fill_dist), axis=1)

left_edges = {'north': 'west', 'east': 'north', 'south':'east', 'west': 'south'}
right_edges = {'north': 'east', 'east':'south', 'south':'west', 'west':'north'}
intersections = []
counts = []

for i in range(len(hpwren_cameras_df)):
    direction = hpwren_cameras_df['direction'][i]
    intersection = []
    c1 = (hpwren_cameras_df['long'][i], hpwren_cameras_df['lat'][i])
    r1 = fov_right[i]
    l1 = fov_left[i]
    if (c1 != None) & (r1 != None) & (l1 != None):
        poly1 = geopandas.GeoSeries(Polygon([c1, r1, l1]))
        for j in range(len(hpwren_cameras_df)):
            c2 = (hpwren_cameras_df['long'][j], hpwren_cameras_df['lat'][j])
            r2 = fov_right[j]
            l2 = fov_left[j]
            if (c2 != None) & (r2 != None) & (l2 != None):
                poly2 = geopandas.GeoSeries(Polygon([c2, r2, l2]))
                if poly1.intersects(poly2)[0] == True:
                    if hpwren_cameras_df['camera_abbrev'][j] != hpwren_cameras_df['camera_abbrev'][i]:
                        intersection.append((hpwren_cameras_df['camera_abbrev'][j], hpwren_cameras_df['direction'][j]))
    if len(intersection) > 0:
        intersections.append(intersection)
    else:
        intersections.append(np.nan)

        
hpwren_cameras_df['intersections'] = intersections

In [165]:
hpwren_cameras_final_df.head()

,camera_id,direction,camera_name,camera_abbrev,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution,y_resolution,center_lat,center_long,center_angle,properties.description.url,intersections
0,hpwren0_unknown direction,unknown direction,NaN,tje,tje-1-mobo-c,-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,NaN,NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html,NaN
1,hpwren1_north,north,Big Black Mountain,bm,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.181599,-116.807554,-0.024816,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, north), (bl, east), (bh, north), (bh, ea..."
2,hpwren1_east,east,Big Black Mountain,bm,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.158781,-116.790230,-0.064085,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bh, east), (bh, south), (cp, north), (cp, ea..."
3,hpwren1_south,south,Big Black Mountain,bm,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.157932,-116.807962,0.065022,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, east), (bl, south), (bh, south), (cp, no..."
4,hpwren1_west,west,Big Black Mountain,bm,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.159091,-116.858706,0.016519,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, north), (bl, east), (bl, south), (bh, so..."


## Select Final Columns

In [166]:
columns = [
    "camera_id",
    "direction",
    "camera_name",
    "camera_abbrev",
    "image_id",
    "long",
    "lat",
    "elevation",
    "geometry.type",
    "geometry.coordinates",
    "x_resolution_manual",
    "y_resolution_manual",
    "center_lat_manual",
    "center_long_manual",
    "center_angle",
    "properties.description.url",
    "intersections"
    # "properties.latest-images",
]
hpwren_cameras_final_df = hpwren_cameras_df[columns]

hpwren_cameras_final_df = hpwren_cameras_final_df.rename(
    columns={
        "x_resolution_manual": "x_resolution",
        "y_resolution_manual": "y_resolution",
        "center_lat_manual": "center_lat",
        "center_long_manual": "center_long",
    },
)

hpwren_cameras_final_df

,camera_id,direction,camera_name,camera_abbrev,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution,y_resolution,center_lat,center_long,center_angle,properties.description.url,intersections
0,hpwren0_unknown direction,unknown direction,NaN,tje,tje-1-mobo-c,-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,NaN,NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html,NaN
1,hpwren1_north,north,Big Black Mountain,bm,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.181599,-116.807554,-0.024816,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, north), (bl, east), (bh, north), (bh, ea..."
2,hpwren1_east,east,Big Black Mountain,bm,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.158781,-116.790230,-0.064085,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bh, east), (bh, south), (cp, north), (cp, ea..."
3,hpwren1_south,south,Big Black Mountain,bm,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.157932,-116.807962,0.065022,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, east), (bl, south), (bh, south), (cp, no..."
4,hpwren1_west,west,Big Black Mountain,bm,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.159091,-116.858706,0.016519,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,"[(bl, north), (bl, east), (bl, south), (bh, so..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,hpwren_missing5_west,west,Santiago Peak,stgo,stgo-w-mobo-c,-117.534115,33.711172,5669.0,NaN,NaN,3072.0,2048.0,33.710503,-117.554501,0.032835,NaN,"[(chino, north), (chino, east), (chino, south)..."
127,hpwren_missing6_north,north,White Star,ws,ws-n-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.650875,-116.317954,-0.016515,NaN,"[(bm, east), (bm, south), (cp, east), (cp, sou..."
128,hpwren_missing6_east,east,White Star,ws,ws-e-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.646405,-116.305941,-0.071166,NaN,"[(pi, east), (ml, east), (ml, south), (mlo, ea..."
129,hpwren_missing6_south,south,White Star,ws,ws-s-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.644742,-116.317985,0.011411,NaN,"[(pi, east), (pi, south), (lp, east), (lp, sou..."


## Write camera metadata to raw and processed folders

In [167]:
# current_time_str = datetime.now(tz=pytz.UTC).strftime("%Y%m%d_%H_%M_%S")

# hpwren cameras only
hpwren_cameras_final_df.to_csv(
    f"../../data/processed/camera_metadata_hpwren.csv", index=False
)